# Yahooo Finance Exploration

This notebook is dedicated to exploring basic yfinance functionalities.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Searching

## Search

We can perform searches on Yahoo Finance using the ```Search``` API:

In [ ]:
search_handler = yf.Search("Apple Inc.")
search_res = search_handler.search()

In [ ]:
quotes_info = [(exch_dict["symbol"], exch_dict["exchDisp"], exch_dict["exchange"]) 
               for exch_dict in search_res.quotes 
               if exch_dict["quoteType"] == "EQUITY"]
print("The company Apple is quoted on the following markets:", quotes_info)

## Screen

We can perform more complex searches using ```*Query``` objects, like ```EquityQuery```.

In [ ]:
query = yf.EquityQuery("and", [
    yf.EquityQuery("eq", ["exchange", "NMS"]),
    yf.EquityQuery("gt", ["percentchange", 3])
    ])

yf.screen(query, sortField = "percentchange", sortAsc = False, size = 1)

There are some [predefined screeners](https://yfinance-python.org/reference/api/yfinance.screen.html#yfinance.screen) available:

In [ ]:
query = yf.PREDEFINED_SCREENER_QUERIES['day_gainers']["query"]
yf.screen(query, size = 1)

## Lookup

The Lookup API allows other search functionalities and presents them in a useful DataFrame format, with methods dedicated to quote types.

In [ ]:
lookup_res = yf.Lookup("Apple Inc.")
lookup_res.all.head()

In [ ]:
# Resulting stocks
print("Stocks:")
lookup_res.stock.head()

In [ ]:
print("ETFs:")
lookup_res.etf.head()

## Downloading Ticker Info

To retrieve info on a single financial instrument, for example [Apple](https://it.finance.yahoo.com/quote/AAPL/):

In [ ]:
ticker = yf.Ticker("AAPL")

We can obtain a lot of data from this Ticker object and its methods. Here are some personal notes on what catches my eyes:

* ```ticker.get_info``` returns a dictionary with miscellaneos info, both anagraphical (e.g. short name, market, exchange, quote type, quote currency, etc.) and market quotes (e.g. day high, volume, bid/ask, etc.);
* ```ticker.get_history_metadata``` returns a dictionary with details on the security's trading operations, e.g. current trading period, recent past trading period hours, etc.;
* ```ticker.get_actions``` returns a DataFrame of past corporate actions, but only Dividends and Stock Splits;
* ```ticker.get_balance_sheet``` returns a DataFrame with a history of the company's balance sheets. Of course this only applies to stocks;
* ```ticker.get_calendar``` returns info on future events, like the next ex-dividend date;
* ```ticker.option_chain``` returns an Options object with info on [all options](https://finance.yahoo.com/quote/AAPL/options/) on this underlying;
* ```ticker.get_news``` returns a list of news on this security.

There a lot of other methods that can be explored [here](https://yfinance-python.org/reference/api/yfinance.Ticker.html#yfinance.Ticker). Unfortunately the API is poorly documented.

In [ ]:
info = ticker.get_info()

name = info["shortName"]
symbol = info["symbol"]
isin = ticker.isin

print(f"Here is info on {name}, symbol {symbol}, of type {info["typeDisp"]}, quoted on {info["fullExchangeName"]} with ISIN code {isin}:")
print(">>", info["longBusinessSummary"])

The info is complex, we can create a ```DataFrame``` to better handle it:

In [ ]:
info_df = pd.DataFrame.from_dict(ticker.get_info(), orient="index")
info_df.columns = ["AAPL"]

info_df.head(10)

We can use the ```Ticker``` to obtain ```Sector``` and ```Industry``` specific info:

In [ ]:
# Ticker to Sector and Industry
sector = yf.Sector(info.get("sectorKey"))
industry = yf.Industry(info.get("industryKey"))

# Sector and Industry to Ticker
tech_ticker = sector.ticker
software_ticker = industry.ticker

# What do we obtain from these new tickers?
df = pd.concat([
    pd.DataFrame.from_dict(tech_ticker.get_info(), orient="index"), 
    pd.DataFrame.from_dict(software_ticker.get_info(), orient="index")
    ], axis=1)
df.columns = ["Sector Ticker", "Industry Ticker"]
df.head(10)

## Downloading Historical Data

We can now download financial data. To import [S&P500](https://finance.yahoo.com/quote/%5EGSPC) daily data from Yahoo Finance, the ticker is "^GSPC":

In [ ]:
df_sp500 = yf.download("^GSPC", start="1900-01-01", multi_level_index=False, auto_adjust=True)
df_sp500.info()

Preliminary data exploration:

In [ ]:
df_sp500.head()

In [ ]:
df_sp500.describe()

The _Open_ column presents some zero values, which is unusual for financial data and probably denotes missing values. Let's investigate:

In [ ]:
zeros= sum(df_sp500["Open"] == 0)
print(f"There are {zeros} zeros in this column:")
df_sp500["Open"].plot()
plt.yscale("log")
plt.ylabel("S&P 500 Open")
plt.show()

Notice that the historical record of opening values is incomplete, but it becomes more reliable starting in the early 1980s, thanks to advancements in trading technology. Much of the pre-1980s data was reconstructed from newspapers, end-of-day reports, or monthly summaries, which often included only high, low, close, and volume. More accurate historical data exists, but it is not available for free in yahoo finance. Missing data is filled with **zero**.

Let's plot _Close_ values, which we expect to be more reliable:

In [ ]:
df_sp500["Close"].plot()
plt.yscale("log")
plt.ylabel("S&P 500 Close")
plt.show()

Historic data is alredy adjusted to account for corporate events (splits), but not for dividends.

To create more complex representations, like candlestick graphs, we can shorten our time series considering only post-1985 data (when open values where recorded):

In [ ]:
candlestick = df_sp500[df_sp500.index > "2000-01-01"]
candlestick.reset_index(inplace=True)

fig = go.Figure(data=[go.Candlestick(x=candlestick["Date"],
                open=candlestick['Open'],
                high=candlestick['High'],
                low=candlestick['Low'],
                close=candlestick['Close'])])

fig.update_layout(
    title=dict(text='S&P 500 Candlestick Graph with Rangeslider'),
    yaxis=dict(
      title=dict(
        text='S&P500 Index'
        )
    )
)

fig.show()